# 3.6 다중 레이블 분류

## 3.6.0 이전 내용 중 필요한 로직

In [1]:
## 기본

import warnings
warnings.filterwarnings('ignore')

# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# MNIST 데이터셋 불러오기
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)

# 데이터셋 분리
X, y = mnist['data'], mnist['target']

# 샘플 데이터 (숫자 5)
some_digit = X[0]

# 레이블 데이터 타입 변환(문자열 -> 숫자)
y = y.astype(np.uint8)

# 테스트 세트 분리
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

<br>

## 3.6.1 다중 레이블 분류(multilabel classification) 시스템

- 분류기가 샘플마다 여러 개의 클래스를 출력해야 할 경우가 있다.
  - ex) 얼굴 인식 분류기
    - 같은 사진에 여러 사람이 등장
    - 인식된 사람마다 하나씩 꼬리표(tag)를 붙여야 한다.
    - 분류기가 앨리스, 밥, 찰리 세 얼굴을 인식하도록 훈련되었다고 가정
    - 분류기가 앨리스와 찰리가 있는 사진을 본다면 `[1, 0, 1]` 을 출력할 것
      - 앨리스 있음(1)
      - 밥 없음(0)
      - 찰리 있음(1)
- 이처럼 여러 개의 이진 꼬리표를 출력하는 분류 시스템을 **다중 레이블 분류(multilabel classification)** 시스템이라고 한다.

<br>

## 3.6.2 다중 레이블 분류기 학습 및 예측

### 3.6.2.1 다중 레이블 생성

- 각 숫자 이미지에 두 개의 타깃 레이블이 담긴 `y_multilabel` 배열 생성
  - `y_train_large`
    - 숫자가 큰 값(7, 8, 9)인 지를 나타냄
    - 홀수인지를 나타냄

In [3]:
y_train_large = (y_train >= 7) # 숫자가 7 이상 인지 아닌지 (7 이상: 1, 7 미만: 0)
y_train_odd = (y_train % 2 == 1) # 홀수인지 아닌지 (홀수: 1, 짝수: 0)
y_multilabel = np.c_[y_train_large, y_train_odd]

<br>

### 3.6.2.2 다중 레이블 분류 지원 알고리즘

- `KNeighborsClassifer` 인스턴스 생성 후 다중 타깃 배열을 사용하여 훈련 시킨다. (`KNeighborsClassifier`는 다중 레이블 분류를 지원)
- 모든 분류기가 다중 레이블 분류를 지원하진 않는다.
- 다중 레이블 분류 지원 알고리즘
  - KNN
  - 결정 트리
  - 랜덤 포레스트
  - `OneVsRestClassifier`

In [2]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)

KNeighborsClassifier()

<br>

### 3.6.2.3 예측

- 예측을 만들면 레이블이 두 개 출력된다.

In [4]:
knn_clf.predict([some_digit])

array([[False,  True]])

- 숫자 5는 크지 않음 (`False`)
- 숫자 5는 홀수임 (`True`)

<br>

## 3.6.3 다중 레이블 분류기 평가

### 3.6.3.1 평가 방법

- 적절한 평가 지표는 프로젝트에 따라 다르다.
  - ex) 각 레이블의 $F_1$ 점수(또는 여러 가지 이진 분류 지표 사용(ex. 정밀도, 재현율, ROC AUC 등))를 구하고 간단하게 **평균 점수 계산**

<br>

### 3.6.3.2 모든 레이블에 대한 $F_1$ 점수의 평균 계산 예시

In [5]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score

y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3)
f1_score = f1_score(y_multilabel, y_train_knn_pred, average='macro')
f1_score

ValueError: average has to be one of (None, 'micro', 'macro', 'weighted', 'samples')

<br>

### 3.6.3.3 레이블 비율에 따른 가중치 부여

- 위의 코드는 모든 레이블의 가중치가 동일하다고 가정한다.
- 만약 앨리스 사진이 밥이나 찰리 사진보다 훨씬 많다면 앨리스 사진에 대한 분류기의 점수에 더 높은 가중치를 둬야 한다.
  - ex) 레이블에 클래스의 **지지도(support, 타깃 레이블에 속한 샘플 수)**를 가중치로 둠
  - 이렇게 하려면 위 코드에서 `average="weighted"`로 설정하면 된다.

<br>

### cf) 사이킷런 평균 계산 방식 및 다중 레이블 분류기 지표

- `average="marco"`
  - 모든 클래스의 FP, FN, TP 총합을 이용해 $F_1$ 점수를 계산  
  
  
- 다음 함수들이 다중 분류를 지원
  - `accuracy_score`
  - `precision_score`
  - `recall_score`
  - `classification_report`